# Domain_walls

### Estimate polarization

In [1]:
import numpy as np
from ase.geometry import find_mic

class PolarizationEstimator:
    """
    This script uses the point charge model and you need to have both optimized CONTCAR and high symmtry structure POSCAR files to determine the polarization.
    """
    def __init__(self, centrosymmetric_structure, ferroelectric_structure, nominal_charges=None, cation_types=None, anion_types=None, ifnormalized=True):
        self.cs_struct = centrosymmetric_structure
        self.fe_struct = ferroelectric_structure
        self.cation_types = set(cation_types) if cation_types else set()
        self.anion_types = set(anion_types) if anion_types else set()
        self.nominal_charges = None
        self.ifnormalized = ifnormalized

        # Validate input
        if nominal_charges is not None:
            self.nominal_charges = np.array(nominal_charges)
            if len(self.nominal_charges) != len(self.cs_struct):
                raise ValueError("Length of nominal charges must match the number of atoms.")
        elif self.cation_types and self.anion_types:
            self._assign_charges_based_on_types()
        else:
            raise ValueError("Either nominal_charges or both cation_types and anion_types must be specified.")

        if len(self.cs_struct) != len(self.fe_struct):
            raise ValueError("The number of atoms in both structures must be the same.")

    def _assign_charges_based_on_types(self):
        """
        Assign nominal charges based on cation and anion types.
        """
        symbols = self.cs_struct.get_chemical_symbols()
        charges = []
        for symbol in symbols:
            if symbol in self.cation_types:
                charges.append(1.0)  # Default cation charge
            elif symbol in self.anion_types:
                charges.append(-1.0)  # Default anion charge
            else:
                raise ValueError(f"Element '{symbol}' not found in cation_types or anion_types.")
        self.nominal_charges = np.array(charges)

    def compute_ionic_polarization(self):
        """
        Compute the ionic polarization direction vector (normalized).
        """
        pos_centro = self.cs_struct.get_positions(wrap=True)
        pos_ferro = self.fe_struct.get_positions(wrap=True)
        cell = self.cs_struct.get_cell()

        displacements = find_mic(pos_ferro - pos_centro, cell=cell)[0]
        dipole_contributions = self.nominal_charges[:, None] * displacements

        total_dipole = dipole_contributions.sum(axis=0)
        volume = self.cs_struct.get_volume()
        polarization_vector = total_dipole / volume
        if self.ifnormalized:
            return polarization_vector / np.linalg.norm(polarization_vector)
        else:
            return polarization_vector


In [2]:
from ase.io import read
ini_atoms = read("../examples/Domain_walls/GeTe/POSCAR")
fin_atoms = read("../examples/Domain_walls/GeTe/CONTCAR")
polarization = PolarizationEstimator(ini_atoms, fin_atoms, cation_types=["Ge"], anion_types=["Te"]).compute_ionic_polarization()
polarization

array([0.57733263, 0.57741119, 0.57730698])

### Build domain wall

In [13]:
from ase import Atoms
from ase.build import cut, rotate, stack
from ase.io import read
from ase.neighborlist import NeighborList
from ase.spacegroup import get_spacegroup
from ase.visualize import view
from numpy.linalg import norm
import io
import numpy as np

class DomainWallSystem:
    def __init__(self, ferroelectric_structure, polarization_vector, domain_angle=90, domain_size=3, stack_axis=2, cutoff=2.5):
        self.atoms = ferroelectric_structure
        self.P1 = polarization_vector
        self.domain_angle = domain_angle
        self.domain_size = domain_size
        self.stack_axis = stack_axis
        self.cutoff = cutoff
        
        self.P1_norm = self.P1 / np.linalg.norm(self.P1)
        self.theta = np.radians(self.domain_angle)
        if not np.isclose(self.P1_norm[0], 0):
            v = np.array([0, 1, 0]) 
        else:
            v = np.array([1, 0, 0])
        u = v - np.dot(v, self.P1_norm) * self.P1_norm
        u = u / np.linalg.norm(u)
        self.P2_norm = np.cos(self.theta) * self.P1_norm + np.sin(self.theta) * u

    def calculate_rotation_matrix(self):
        dot_product = np.round(np.dot(self.P1_norm, self.P2_norm), 6)
        
        if np.isclose(dot_product, 1.0): 
            return np.eye(3) 
        elif np.isclose(dot_product, -1.0):
            if not np.isclose(self.P1_norm[0], 0):
                R = np.array([-self.P1_norm[1], self.P1_norm[0], 0])
            else:
                R = np.array([0, -self.P1_norm[2], self.P1_norm[1]])
        else:
            R = np.cross(self.P1_norm, self.P2_norm)
            
        R = R / np.linalg.norm(R)
        K = np.array([[0, -R[2], R[1]], [R[2], 0, -R[0]], [-R[1], R[0], 0]])
        I = np.eye(3)
        return I + np.sin(self.theta) * K + (1 - np.cos(self.theta)) * np.dot(K, K)

    def redifined_cell(self, atoms, new_cell):
        atoms.set_cell(new_cell, scale_atoms=False)
        positions = atoms.get_positions()
        cell = atoms.get_cell()
        new_positions = np.dot(positions, np.linalg.inv(cell))
        atoms.set_positions(np.dot(new_positions, new_cell))
        atoms.center()
        return atoms

    def remove_atoms_outside(self, atoms):        
        positions = atoms.get_positions()
        cell = atoms.get_cell()
        new_positions = np.dot(positions, np.linalg.inv(cell))
        mask = np.all((new_positions >= 0) & (new_positions < 1), axis=1)
        atoms = atoms[mask]
        return atoms

    def remove_close_atoms(self, atoms):
        nl = NeighborList([self.cutoff / 2.0] * len(atoms), self_interaction=False, bothways=True)
        nl.update(atoms)
        indices_to_remove = set()
        for i in range(len(atoms)):
            indices, offsets = nl.get_neighbors(i)
            for idx in indices:
                if i < idx:
                    distance = atoms.get_distance(i, idx)
                    if distance < self.cutoff:
                        indices_to_remove.add(idx)
        atoms = atoms[[atom.index not in indices_to_remove for atom in atoms]]
        return atoms

    def calculate_lattice_strain(self, slab1, slab2):
        a1, b1, c1 = slab1.cell
        a2, b2, c2 = slab2.cell
        strain_a = (norm(a1) - norm(a2)) / norm(a2) * 100
        strain_b = (norm(b1) - norm(b2)) / norm(b2) * 100
        strain_c = (norm(c1) - norm(c2)) / norm(c2) * 100
        print("Lattice strain for DW:")
        print(f"strain along a (%): {strain_a:.2f}")
        print(f"strain along b (%): {strain_b:.2f}")
        print(f"strain along c (%): {strain_c:.2f}")

    def generate_domain_structure(self):
        """
        Generate a single-domain structure with a specified domain wall angle.
        """
        supercell_matrix = [1, 1, 1]
        supercell_matrix[self.stack_axis] = self.domain_size
        slab1 = self.atoms.copy() * supercell_matrix
        slab1.center()

        rotation_matrix = self.calculate_rotation_matrix()
        slab2 = self.atoms.copy() * (self.domain_size*2, self.domain_size*2, self.domain_size*2)
        slab2.positions = np.dot(slab2.positions, rotation_matrix.T)
        slab2.cell = np.dot(slab2.cell, rotation_matrix.T)
        slab2.center()

        slab2 = self.redifined_cell(slab2, slab1.cell.copy())
        slab2 = self.remove_atoms_outside(slab2)

        stacked_cell = slab1.cell.copy()
        stacked_cell[self.stack_axis, self.stack_axis] = slab1.cell[self.stack_axis, self.stack_axis] + slab2.cell[self.stack_axis, self.stack_axis] + 1
        slab2.positions[:, self.stack_axis] += slab1.cell[self.stack_axis, self.stack_axis] + 1
        stacked_positions = np.vstack([slab1.positions, slab2.positions])
        
        stacked_slab = Atoms(
            symbols=slab1.get_chemical_symbols() + slab2.get_chemical_symbols(),
            positions=stacked_positions,
            cell=stacked_cell,
            pbc=True
        )
        self.remove_close_atoms(stacked_slab)
        
        return stacked_slab

In [14]:
stacked_slab = DomainWallSystem(fin_atoms, [0.57733263, 0.57741119, 0.57730698]).generate_domain_structure()
view(stacked_slab)

<Popen: returncode: None args: ['C:\\Users\\wangchangrui\\.conda\\envs\\Q-Ap...>